<a href="https://colab.research.google.com/github/chongzicbo/nlp-ml-dl-notes/blob/master/code/pytorch_tutorials/pytorch05%EF%BC%9A%E5%BA%8F%E5%88%97%E6%A8%A1%E5%9E%8B%E5%92%8CLSTM%E7%BD%91%E7%BB%9C(%E9%95%BF%E7%9F%AD%E8%AE%B0%E5%BF%86%E7%BD%91%E7%BB%9C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

之前我们已经学过了许多的前馈网络. 所谓前馈网络, 就是网络中不会保存状态. 然而有时 这并不是我们想要的效果. 在自然语言处理 (NLP, Natural Language Processing) 中, 序列模型是一个核心的概念. 所谓序列模型, 即输入依赖于时间信息的模型. 一个典型的序列模型是隐马尔科夫模型 (HMM, Hidden Markov Model). 另一个序列模型的例子是条件随机场 (CRF, Conditional Random Field).

循环神经网络是指可以保存某种状态的神经网络. 比如说, 神经网络中上个时刻的输出可以作为下个 时刻的输入的一部分, 以此信息就可以通过序列在网络中一直往后传递. 对于LSTM (Long-Short Term Memory) 来说, 序列中的每个元素都有一个相应的隐状态 $h_t$, 该隐状态 原则上可以包含序列当前结点之前的任一节点的信息. 我们可以使用隐藏状态来预测语言模型 中的单词, 词性标签以及其他各种各样的东西.

# Pytorch中的LSTM
在正式学习之前，有几个点要说明一下，Pytorch中LSTM的输入形式是一个3D的Tensor，每一个维度都有重要的意义，第一个维度就是序列本身，第二个维度是mini-batch中实例的索引，第三个维度是输入元素的索引，我们之前没有接触过mini-batch，所以我们就先忽略它并假设第二维的维度是1。

如果要用"The cow jumped"这个句子来运行一个序列模型，那么就应该把它整理成如下的形式：
$$
\( \begin{split} \begin{bmatrix} \overbrace{q\text{The}}^\text{row vector} \ q\text{cow} \ q_\text{jumped} \end{bmatrix} \end{split} \)
$$
除了有一个额外的大小为1的第二维度.

此外, 你还可以向网络逐个输入序列, 在这种情况下, 第一个轴的大小也是1.

来看一个简单的例子.

In [4]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
torch.manual_seed(1)

In [5]:
lstm=nn.LSTM(3,3) #输入维度为3，输出维度为3
inputs=[torch.randn(1,3) for _ in range(5)] #生成一个长度为5的序列
#初始化隐藏状态
hidden=(torch.randn(1,1,3),torch.randn(1,1,3))

for i in inputs:
  #将序列中的元素逐个输入到LSTM，经过每步操作，hidden的值包含了隐藏状态的信息
  out,hidden=lstm(i.view(1,1,-1),hidden)


# 另外我们可以对一整个序列进行训练.
# LSTM第一个返回的第一个值是所有时刻的隐藏状态
# 第二个返回值是最后一个时刻的隐藏状态
#(所以"out"的最后一个和"hidden"是一样的)
# 之所以这样设计:
# 通过"out"你能取得任何一个时刻的隐藏状态，而"hidden"的值是用来进行序列的反向传播运算, 具体方式就是将它作为参数传入后面的 LSTM 网络.

# 增加额外的第二个维度.
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # 清空隐藏状态. 
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>)
(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<StackBackward>))


# 例子:用LSTM来进行词性标注
在这部分, 我们将会使用一个 LSTM 网络来进行词性标注. 在这里我们不会用到维特比算法, 前向-后向算法或者任何类似的算法,而是将这部分内容作为一个 (有挑战) 的练习留给读者, 希望读者在了解了这部分的内容后能够实现如何将维特比算法应用到 LSTM 网络中来.

该模型如下:输入的句子是$w_1,\ldots,w_M$,对应的词性为$y_1,\ldots,y_i$,用$\hat{y}_i$表示对$w_i$的词性的预测，标签的集合为$T$。
这是一个结构预测模型, 我们的输出是一个序列$\hat{y}_1,\ldots,\hat{y}_i$,其中$\hat{y}_i \in T$。

在进行预测时, 需将句子每个词输入到一个 LSTM 网络中. 将时刻ii的隐藏状态标记为$h_i$,同样地, 对每个标签赋一个独一无二的索引 (类似 word embeddings 部分 word_to_ix 的设置). 然后就得到了$\hat{y}_i$的预测规则。
$$
\hat{y}^i=argmaxj(logSoftmax(Ahi+b))j
$$

即先对隐状态进行一个仿射变换, 然后计算一个对数 softmax, 最后得到的预测标签即为对数 softmax 中最大的值对应的标签. 注意, 这也意味着 AA 空间的维度是 |T|.

准备数据:


In [6]:
def prepare_sequence(seq,to_ix):
  idxs=[to_ix[w] for w in seq]
  return torch.tensor(idxs,dtype=torch.long)

In [7]:
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix={}
for sent,tags in training_data:
  for word in sent:
    if word not in word_to_ix:
      word_to_ix[word]=len(word_to_ix)

print(word_to_ix)
tag_to_ix={"DET":0,"NN":1,"V":2}  
EMBEDDING_DIM=6
HIDDEN_DIM=6    

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


## 创建模型

In [9]:
class LSTMTagger(nn.Module):
  def __init__(self,embedding_dim,hidden_dim,vocab_size,tagset_size):
    super(LSTMTagger,self).__init__()
    self.hidden_dim=hidden_dim
    self.word_embeddings=nn.Embedding(vocab_size,embedding_dim)

    #LSTM以word_embedding作为输入，输出维度为hidden_dim的隐藏状态
    self.lstm=nn.LSTM(embedding_dim,hidden_dim)

    #线性层将隐藏状态空间映射到标注空间
    self.hidden2tag=nn.Linear(hidden_dim,tagset_size)
    self.hidden=self.init_hidden()

  def init_hidden(self):
    # 一开始并没有隐藏状态所以我们要先初始化一个
    # 关于维度为什么这么设计请参考Pytoch相关文档
    # 各个维度的含义是 (num_layers, minibatch_size, hidden_dim)
    return (torch.zeros(1,1,self.hidden_dim),torch.zeros(1,1,self.hidden_dim))

  def forward(self,sentence):
    embeds=self.word_embeddings(sentence)
    lstm_out,self.hidden=self.lstm(embeds.view(len(sentence),1,-1),self.hidden)
    tag_space=self.hidden2tag(lstm_out.view(len(sentence),-1))
    tag_space=F.log_softmax(tag_space,dim=1)
    return tag_space

## 训练模型

In [14]:
model=LSTMTagger(EMBEDDING_DIM,HIDDEN_DIM,len(word_to_ix),len(tag_to_ix))
loss_function=nn.NLLLoss()
optimizer=optim.SGD(model.parameters(),lr=0.1)
# 查看训练前的分数
# 注意: 输出的 i,j 元素的值表示单词 i 的 j 标签的得分
# 这里我们不需要训练不需要求导，所以使用torch.no_grad()

with torch.no_grad():
  inputs=prepare_sequence(training_data[0][0],word_to_ix)
  tag_scores=model(inputs)
  print(tag_scores)

tensor([[-1.3852, -0.9624, -1.0003],
        [-1.3570, -1.1749, -0.8354],
        [-1.3794, -1.2678, -0.7618],
        [-1.3699, -1.1893, -0.8177],
        [-1.3667, -1.2508, -0.7792]])


In [12]:
for x in model.parameters():
  print(x.shape)

torch.Size([9, 6])
torch.Size([24, 6])
torch.Size([24, 6])
torch.Size([24])
torch.Size([24])
torch.Size([3, 6])
torch.Size([3])


In [15]:
for epoch in range(300):
  for sentence,tags in training_data:
    #梯度清零
    model.zero_grad()
    #清空LSTM的隐藏状态,将其从上个实例的历史中分离出来
    model.hidden=model.init_hidden()

    sentence_in=prepare_sequence(sentence,word_to_ix)
    targets=prepare_sequence(tags,tag_to_ix)

    #前向传播
    tag_scores=model(sentence_in)

    #计算损失和梯度值
    loss=loss_function(tag_scores,targets)
    loss.backward()
    #更新梯度
    optimizer.step()


In [16]:
with torch.no_grad():
  inputs=prepare_sequence(training_data[0][0],word_to_ix)
  tag_scores=model(inputs)
  # 句子是 "the dog ate the apple", i,j 表示对于单词 i, 标签 j 的得分.
  # 我们采用得分最高的标签作为预测的标签. 从下面的输出我们可以看到, 预测得
  # 到的结果是0 1 2 0 1. 因为 索引是从0开始的, 因此第一个值0表示第一行的
  # 最大值, 第二个值1表示第二行的最大值, 以此类推. 所以最后的结果是 DET
  # NOUN VERB DET NOUN, 整个序列都是正确的!
  print(tag_scores)

tensor([[-0.0996, -2.9313, -3.1838],
        [-5.1570, -0.0110, -5.2682],
        [-3.9731, -4.3701, -0.0320],
        [-0.0234, -4.4200, -4.5014],
        [-4.6125, -0.0127, -5.9011]])
